In [1]:
import torch
#print(torch.__version__)
#print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.device_count())  

True
4


In [2]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import json

def load_data(datafile):
    """Loads the regular and simplified news articles and appends them as
    dictionnaries into a list.
    Returns:
    all data as a list of dictionaries
    """
    all_data = []
    
    with open(datafile, 'r') as data:
        data = json.load(data)
        for datapoint in data:
            entry = {
                "text": datapoint['original']['article'],
                "lätt text": datapoint['simplified']['article']
            }
            all_data.append(entry)
    return all_data

In [3]:
training_data, test_data = train_test_split(load_data("data.json"), test_size=0.1)        # splitting the data

# creating datasets 

train_dataset = Dataset.from_list(training_data)
test_dataset = Dataset.from_list(test_data)
print(train_dataset[1])

{'text': 'Filip & Fredriks dokumentär ”Den sista resan” om en roadtrip genom Europa är Sveriges bidrag till Oscarsgalan. – Helt otroligt, säger Filip Hammar under presskonferensen. ”Den sista resan” följer Filip Hammars pappa Lars – som efter 40 år som älskad fransklärare i Köping går i pension. När han blir passiv och viljelös tar sonen Filip och bästa vännen Fredrik med Lars till hans älskade Frankrike för att tända livsgnistan igen. Filmen har varit en publiksuccé i Sverige och lockat flera hundratusentals besökare till biosalongerna.Nu kan ”Den sista resan” ta sig hela vägen till Oscarsgalan. – Det är helt otroligt att få vara med i ett sådant här sammanhang. Men också faktumet att det handlar om min familj, och där inkluderar jag även Fredrik, att det var en sådan personlig historia, säger Filip Hammar under presskonferensen. ”Dokumentären sticker ut” Oscarsgalan nästa år hålls på Dolby Theatre in Hollywood den 2 mars och blir den 97:e gången som världens finaste filmpris delas ut

In [4]:
from transformers import AutoTokenizer

checkpoint = "./final-task-fine-tuned-model-40k-traindata/checkpoint-4000"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)   #legacy=False, use_fast=False) 

You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/home/gusniemian@GU.GU.SE/.local/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
prefix = "förenkla språket: "
def preprocess_function(examples):
    """ Preprocesses the data and adds the prefix.
        Tokenizes the data with mt5-tokenizer.
    """   
    encoding = tokenizer(
    [prefix + sequence for sequence in examples["text"]],
    padding="longest",
    max_length=1200,
    truncation=True,
    return_tensors="pt")

    input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

    # targets tokenizing
    target_encoding = tokenizer(
        [sequence for sequence in examples["lätt text"]],
        padding="longest",
        max_length=600,
        truncation=True,
        return_tensors="pt",
    )
    labels = target_encoding.input_ids

    # replacing padding token id's of the labels by -100 so it's ignored by the loss
    labels[labels == tokenizer.pad_token_id] = -100
    
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [6]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

In [7]:
print(tokenized_train_dataset[1])

{'text': 'Filip & Fredriks dokumentär ”Den sista resan” om en roadtrip genom Europa är Sveriges bidrag till Oscarsgalan. – Helt otroligt, säger Filip Hammar under presskonferensen. ”Den sista resan” följer Filip Hammars pappa Lars – som efter 40 år som älskad fransklärare i Köping går i pension. När han blir passiv och viljelös tar sonen Filip och bästa vännen Fredrik med Lars till hans älskade Frankrike för att tända livsgnistan igen. Filmen har varit en publiksuccé i Sverige och lockat flera hundratusentals besökare till biosalongerna.Nu kan ”Den sista resan” ta sig hela vägen till Oscarsgalan. – Det är helt otroligt att få vara med i ett sådant här sammanhang. Men också faktumet att det handlar om min familj, och där inkluderar jag även Fredrik, att det var en sådan personlig historia, säger Filip Hammar under presskonferensen. ”Dokumentären sticker ut” Oscarsgalan nästa år hålls på Dolby Theatre in Hollywood den 2 mars och blir den 97:e gången som världens finaste filmpris delas ut

In [8]:
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [10]:
from evaluate import load
import numpy as np

sari = load("sari")
rouge = load("rouge")

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred  
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    sources = [example["text"] for example in test_dataset]  
    references = [[example["lätt text"]] for example in test_dataset] 
    
    print("sources",sources)
    print("predictions",decoded_preds)
    print("references",references)

    # calculating SARI
    sari_score = sari.compute(sources=sources, predictions=decoded_preds, references=references)

    return {
        "sari": sari_score,
    }


In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [13]:

for param in model.parameters():
    if not param.requires_grad:
        param.requires_grad = True


In [14]:
training_args = Seq2SeqTrainingArguments(
    output_dir="final-language-fine-tuned-model",
    eval_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8, # for memory issues
    weight_decay=0.01, #to prevent overfitting
    save_total_limit=2,
    num_train_epochs=20,
    overwrite_output_dir=True,
    predict_with_generate=True,
    logging_strategy="steps",
    fp16=True, 
    warmup_steps=50, 
    push_to_hub=False,
)


In [15]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
print(f"Training on {device}")


Training on cuda


In [16]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.12/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [17]:
trainer.train()

/usr/local/lib64/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Sari
0,No log,3.270493,{'sari': 33.62571951103959}
2,No log,3.244759,{'sari': 33.71721819040121}
4,No log,3.186974,{'sari': 33.535337180127065}
6,No log,3.116630,{'sari': 32.7505505166583}
8,No log,3.051745,{'sari': 33.282336954903556}
10,No log,3.007188,{'sari': 35.10583314679875}
12,No log,2.982897,{'sari': 35.55144491812662}
13,No log,2.976548,{'sari': 35.77589931524155}


/home/gusniemian@GU.GU.SE/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Trainer is attempting to log a value of "{'sari': 33.62571951103959}" of type <class 'dict'> for key "eval/sari" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


sources ['Fyra av tio svenskar tror på någon form av paranormala fenomen. Och bland kvinnor är det så många som hälften som tror på det övernaturliga. Det visar en ny studie vid Uppsala universitet. Cristoffer Tidelius är doktor i religionssociologi vid Teologiska institutionen vid Uppsala universitet, och han skrev nyligen en avhandling om hur vanligt det är att svenskar tror på paranormala fenomen, alltså saker som inte går att förklara vetenskapligt. Studien visar att 41 procent av vuxna svenskar tror på paranormala fenomen. Skillnaden mellan män och kvinnor är stor: 37 procent av männen tror, och 52 procent av kvinnorna. Samtidigt är det vanligare att män tror på ufo:n, det vill säga oidentifierade flygande föremål. Mer synligt i sociala medier När det gäller antalet som tror på det övernaturliga är det inte helt enkelt att göra en historisk jämförelse, då det är svårt att jämföra resultaten med tidigare studier. – Jag har dock inte fått en indikation på att det växer lavinartat. D

Trainer is attempting to log a value of "{'sari': 33.62571951103959}" of type <class 'dict'> for key "eval/sari" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


...

Trainer is attempting to log a value of "{'sari': 35.55144491812662}" of type <class 'dict'> for key "eval/sari" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


sources ['Fyra av tio svenskar tror på någon form av paranormala fenomen. Och bland kvinnor är det så många som hälften som tror på det övernaturliga. Det visar en ny studie vid Uppsala universitet. Cristoffer Tidelius är doktor i religionssociologi vid Teologiska institutionen vid Uppsala universitet, och han skrev nyligen en avhandling om hur vanligt det är att svenskar tror på paranormala fenomen, alltså saker som inte går att förklara vetenskapligt. Studien visar att 41 procent av vuxna svenskar tror på paranormala fenomen. Skillnaden mellan män och kvinnor är stor: 37 procent av männen tror, och 52 procent av kvinnorna. Samtidigt är det vanligare att män tror på ufo:n, det vill säga oidentifierade flygande föremål. Mer synligt i sociala medier När det gäller antalet som tror på det övernaturliga är det inte helt enkelt att göra en historisk jämförelse, då det är svårt att jämföra resultaten med tidigare studier. – Jag har dock inte fått en indikation på att det växer lavinartat. D

/home/gusniemian@GU.GU.SE/.local/lib/python3.12/site-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(
/home/gusniemian@GU.GU.SE/.local/lib/python3.12/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib64/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Trainer is attempting to log a value of "{'sari': 35.77589931524155}" of type <class 'dict'> for key "eval/sari" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we droppe

sources ['Fyra av tio svenskar tror på någon form av paranormala fenomen. Och bland kvinnor är det så många som hälften som tror på det övernaturliga. Det visar en ny studie vid Uppsala universitet. Cristoffer Tidelius är doktor i religionssociologi vid Teologiska institutionen vid Uppsala universitet, och han skrev nyligen en avhandling om hur vanligt det är att svenskar tror på paranormala fenomen, alltså saker som inte går att förklara vetenskapligt. Studien visar att 41 procent av vuxna svenskar tror på paranormala fenomen. Skillnaden mellan män och kvinnor är stor: 37 procent av männen tror, och 52 procent av kvinnorna. Samtidigt är det vanligare att män tror på ufo:n, det vill säga oidentifierade flygande föremål. Mer synligt i sociala medier När det gäller antalet som tror på det övernaturliga är det inte helt enkelt att göra en historisk jämförelse, då det är svårt att jämföra resultaten med tidigare studier. – Jag har dock inte fått en indikation på att det växer lavinartat. D

TrainOutput(global_step=20, training_loss=4.654166030883789, metrics={'train_runtime': 224.8879, 'train_samples_per_second': 4.002, 'train_steps_per_second': 0.089, 'total_flos': 629997216933888.0, 'train_loss': 4.654166030883789, 'epoch': 13.333333333333334})

In [18]:
text1 = "En allvarlig trafikolycka inträffade i Brändön utanför Luleå natten mot söndagen. Tre unga män dog i olyckan och en kvinna i övre tonåren vårdas med livshotande skador. Polisen utreder olyckan. – Vi har inte kunna prata med den unga kvinnan som vårdas på sjukhus, säger polisens förundersökningsledare Fredrik Lampinen. Strax efter klockan 01 under natten till söndagen kom larm om en singelolycka i Brändön där flera personer varit inblandade. När polisen kom fram till platsen visade det sig att bilen voltat. – Det handlar om en allvarlig olycka, sa polisens presstalesperson Maria Linné. Tre unga män i övre tonårsåldern omkom i samband med olyckan. En kvinna, även hon i övre tonåren, vårdas på sjukhus med livshotande skador. Samtliga är hemmahörande i Luleå kommun. Utreds som vållande till annans död Polisen utreder händelsen. Olycksplatsen har dokumenterat och bilen har tagits i beslag för teknisk undersökning. Olyckan utreds som vållande till annans död, vållande till kroppsskada och vårdslöshet i trafik. Ingen är delgiven misstanke om brott. – Vi har hållit flera förhör men vi har inte kunna prata med den unga kvinnan som vårdas på sjukhus. Ingen är delgiven misstanke om brott, säger Erik Lampinen. Många sörjande Under söndagen var POSOM-gruppen på plats i Porsö kyrka dit de som ville kunde komma för stöd och råd. POSOM är en förkortning av Psykologiskt och socialt omhändertagande. Under söndagseftermiddagen sökte sig många till olycksplatsen för att sörja och tända ljus – tidvis så många att det var svårt att ta sig fram i trafiken."

In [19]:
text2 = "Misstänkt terrorattack i Ankara En eller flera explosioner och skottlossning har inträffat på företaget Turkish Aerospace Industries högkvarter utanför Ankara, rapporterar turkiska medier på onsdagen. Turkiets inrikesminister skriver på X att det rör sig om en terrorattack med flera döda. Attacken mot Turkiets teknikcentrum för flyg- och rymdsystem (Tusaş) inträffade bland annat vid entrén till anläggningen. Polis och räddningstjänst är på plats. Enligt Turkiets president Recep Tayyip Erdogan dödades fyra personer i attacken och 14 är skadade, varav tre allvarligt. Inrikesminister Ali Yerlikaya rapporterar att även två gärningspersoner, en kvinna och en man, har ”neutraliserats”. ”Vår kamp kommer att fortgå med beslutsamhet tills den sista terroristen är neutraliserad.” skriver Yerlikaya vidare. Samtidigt rapporterar lokala medier att attacken utfördes av tre terrorister. Dessutom ska personal ha tagits som gisslan, men uppgifterna är inte bekräftade. Explosion fångad på film Videor på en explosion och skottlossning sprids på sociala medier. Flera av klippen, som verifierats av Reuters, visar hur människor tar skydd bakom bilar på den anslutande parkeringen. På bilder från bevakningskameror syns män med automatgevär gå in i byggnaden. Natos generalsekreterare Mark Rutte har kommenterat händelsen och meddelar att ”Nato står bakom vår allierade Turkiet”. Han tillägger i sitt uttalande att organisationen ”starkt fördömer terrorism i alla dess former”. Även Sveriges utrikesminister Maria Malmer Stenergard följer situationen: ”Oroväckande rapporter från Turkiet om döda och skadade i vad som verkar vara ett terrordåd.” skriver ministern på X. Turkish Aerospace Industries Tusaş, var högkvarter ligger nordväst om huvudstaden, arbetar med utveckling och tillverkning av flyg- och rymdsystem. Företaget, som har kopplingar till landets militär, tillverkar bland annat stridsflygplan, obemannade flygfarkoster och attackhelikoptrar."

In [32]:
text3 = "Södra och östra Spanien har drabbats av kraftiga översvämningar som svept med sig bilar, lagt hus under vatten och fått ett tåg att spåra ur. Minst 62 personer har dött i översvämningarna, enligt Reuters. – Av respekt för familjerna kommer vi inte att lämna några ytterligare uppgifter, säger Carlos Mazón, Valencias president. Översvämningarna som orsakats av kraftiga regn och stormar har lett till stora skador från Málagaprovinsen i södra Spanien till Valenciaprovinsen i öster. Minst 62 personer har dött efter regnkaoset, samtliga i Valenciaprovinsen, enligt lokala myndigheter, och flera uppges fortfarande vara saknade. ”Enorma mängder regn” Den meteorologiska myndigheten Aemet har utfärdat en röd varning i de drabbade områdena, vilket är den högsta nivån, rapporterar Málaga Hoy. Värst drabbat är Valenciaprovinsen där vissa områden fick 200 millimeter nederbörd på bara ett par timmar, enligt Aemet. I genomsnitt faller 77 millimeter regn under hela oktober månad i området. – Det här är enorma mängder regn, sa Rubén del Campo, talesperson för Aemet, rapporterar New York Times. Och regnet väntas fortsätta under onsdagen och torsdagen. – Det kan komma över till 100 millimeter i delar av östra Spanien, på bara tolv timmar, säger SVT:s meteorolog Nils Holmqvist. Många fast i sina hem I närheten av Málaga har ett höghastighetståg spårat ur med nära 300 personer ombord. Ingen ska ha skadats enligt spanska myndigheter. Enligt det spanska public service-bolaget RTVE saknas en lastbilschaufför i staden Alcúsia i Valenciaprovinsen samtidigt som ett flertal personer är fast i sina hem. Strömmen ligger också nere på flera håll och personer uppges vara strandsatta på bil- och hustak. I de östra delarna av Spanien saknas fortfarande dussintals personer, enligt BBC, och räddningspersonal söker av området med hjälp av drönare. Arméns räddningsstyrka inkallad Polis och räddningstjänst arbetar med att lyfta människor ut ur översvämmade hus med hjälp av helikoptrar. Arméns räddningsstyrka har kallats in för förstärkning. Regering upprättade på tisdagskvällen en kriskommitté med uppdrag att vidta nödvändiga åtgärder. Spaniens premiärminister Pedro Sánchez skriver i ett inlägg på X att han följer utvecklingen ”med stor oro”. ”Var försiktiga och undvik onödiga resor”, fortsätter han."

In [38]:
text4 = "Stora delar av Gamla stan har under onsdagsmorgonen varit avspärrade efter att en handgranat hittats på Skeppsbron. – Det såg ut som en sådan grönsvart klassisk handgranat, säger ett vittne som var på plats. Larmet kom till polisen vid kvart över sju på onsdagsmorgonen. – Det är en privatperson som hittat ett misstänkt farligt föremål. Vi har spärrat av stora delar av Gamla stan, säger Ola Österling, presstalesperson på Stockholmspolisen. Stora delar av Gamla stan spärrades av vilket hade en stor påverkan på kollektivtrafiken i området, både för bussar och pendelbåt. Enligt uppgifter till Expressen var handgranaten riktad mot en restaurang. Vittnet: ”Grönsvart klassisk handgranat” Emil Glenngård jobbar i området och gick förbi föremålet innan polisen hann sätta upp avspärrningarna på onsdagsmorgonen. – Det såg ut som en sådan grönsvart klassisk handgranat, den låg vid ett övergångsställe. Det var en speciell morgon att vakna, och hamna mitt i epicentret, säger han. – Jag såg föremålet och tänkte ”va fan”. Bomskyddet på plats Efter att det nationella bombskyddet undersökt föremålet kunde polisen bekräfta att det rörde sig om en handgranat. – Vi bedömer att det är en skarp handgranat som ligger på vägbanan i Gamla stan, sa Ola Österling till SVT. Strax efter nio på förmiddagen detonerade bombskyddet handgranaten."

In [5]:
text5 = "Billigare snus och dyrare cigaretter. Så väntas ufallet bli av de skatteförändringar som träder i kraft 1 november. – Jag tror försäljningen kommer öka, säger Mindy Allnor, snusförsäljare i Göteborg. Sammantaget uppgår skattesänkningen på snus till omkring 20 procent. I samma veva höjs skatten på cigaretter med nio procent. Enligt regeringen handlar det om att spegla de hälsofarliga effekterna av snus och cigaretter – där de senare anses vara betydligt mycket värre för hälsan. När SVT besökte en snusbutik i Göteborg på fredagen var de flesta kunderna positiva. – Väldigt skönt för oss som är fast i det här, säger kunden Felix Llado. – Rent allmänt är det inte bra, nikotin är ju inte bra för oss. Men snuset är att föredra framför cigaretterna tänker jag, säger kunden Hans Segerham. Leverantörer bestämmer priser För flera av de vanligaste märkena motsvarar skattesänkningen omkring 55 kronor för en stock (ett tiopack) lössnus, enligt Snusbolagets uträkning. För portionssnus kan det det röra sig om mellan 20 och 30 kronors sänkning per stock. Samtidigt bestämmer varje leverantör sina egna priser – och att alla kommer att sänka priset är inte självklart."

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("./final-language-fine-tuned-model/checkpoint-20")
inputs = tokenizer(text5, return_tensors="pt").input_ids

/home/gusniemian@GU.GU.SE/.local/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [7]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("./final-language-fine-tuned-model/checkpoint-20")
generated_ids = model.generate(
    inputs,
    do_sample=False,
    repetition_penalty=2.0,
    max_new_tokens=300,
    min_new_tokens=200,   # min length for generation
    num_beams=5,     # n beam search
    no_repeat_ngram_size=3,  # prevent repetitive n-grams
)

In [8]:
tokenizer.decode(generated_ids[0], skip_special_tokens=True)

'<extra_id_0> är billigare snus och dyrare cigaretter. För flera av de vanligaste märkena motsvarar skattesänkningen omkring 20 procent för en stock (ett tiopack) lössnus. Enligt Snusbolaget uträkningen handlar det om att spegla de hälsofarliga effekterna av snus till omkring 30 procent. – Det är så bra för oss som är fast i det här, säger Mindy Allnor. De flesta kunderna tycker det är mycket värre för hälsan. På fredagen väntas ufallet bli av de skatteförändringar som träder i kraft 1 november. I samma veva höjs skatten på snus med nio procent. Det finns alltså inte bra för människor. Men snuset kan det röra sig om mellan 20 och 30 kronor per stock. Den första tillverkare bestämmer priserna.'